<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png"/>

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   

# Ajustar un modelo pre-entrenado para la clasifición de textos (PyTorch)

La **clasificación de textos** es una aplicación muy popular de PLN cuyo objetivo es clasificar un texto con una categoría o etiqueta predefinida. Podemos distinguir entre la **clasificación binaria** donde únicamente hay dos clases, o multiclase donde el conjunto de clases predefinidas son tres o más. La diferencia entre **multiclase** y **multilabel** es que en la primera tarea, un texto es clasificada con una única clase, mientras que en la segunda, un texto puede clasificar con varias etiquetas.

En este notebook , estudiaremos cómo ajustar un modelo transformer, en concreto BERT, para la tarea de clasificación de textos. 

Las ventajas de utilizar un modelo pre-entrenado son las siguientes:
- menor coste computacional (vectores ya están pre-entrenados)
- reducen el tiempo y esfuerzo porque puedes utlizar modelos sin tener que entrenar desde cero. 


HuggingFace incluye muchísimos modelos pre-entrenados para muchas tareas de NLP. 

## ¿Qué es fine-tuning?


Fine-tuning es el proceso de usar un modelo pre-entrenado y entrenarlo sobre un dataset para una tarea concreta, como por ejemplo, clasificación de textos o NER. 


Fuente:
https://huggingface.co/docs/transformers/training

Instalamos las librerías que vamos a necesitar:

In [ ]:
!pip install transformers datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Data

Como dataset vamos a utilizar uno proporcionado por HuggingFace, **trec** (https://huggingface.co/datasets/trec), que es una colección de preguntas que han sido clasificadas por el tipo de respuesta que espera. En concreto, las clases son las siguientes (se recogen en el campo **coarse_label**). 

- 'ABBR' (0): la respuesta esperada es una abreviatura.
- 'ENTY' (1): la respuesta esperada es una entidad.
- 'DESC' (2): la respuesta esperada es una descripción.
- 'HUM' (3): la respuesta esperada es una persona.
- 'LOC' (4): la respuesta esperada es un lugar.
- 'NUM' (5): la respuesta esperada es un valor numérico.

El dataset incluye un segundo campo, **fine_label**, donde se da una clasificación más fina del tipo de respuesta esperada para cada pregunta. En este notebook, nos centraremos únicamente en la clasificación basada en **coarse_label**.

El dataset es distribuido con dos splits: train y test.



In [ ]:
from datasets import load_dataset
dict_dataset = load_dataset("trec")
dict_dataset


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 500
    })
})

Mostramos las clases asociadas con coarse_label:

In [ ]:
TARGET_LABELS = dict_dataset['train'].features['coarse_label'].names


Borramos el campo **fine_label**, porque no lo vamos a usar, y renombramos **coarse_label** a **label**

In [ ]:
dict_dataset = dict_dataset.remove_columns(['fine_label'])
dict_dataset = dict_dataset.rename_column('coarse_label','label')

dict_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})

El dataset contiene 5452 instancias para training, y 500 para test. 
Utilizaremos este dataset porque es pequeño y nos permitirá entrenar el modelo en poco tiempo. En HuggingFace, puedes encontrar otros datasets para trabajar en la tarea de clasificación de textos. Por ejemplo, **yelp** (https://huggingface.co/datasets/yelp_review_full) o https://huggingface.co/datasets/sst2

También puedes cargar tu propio dataset desde ficheros en local, como hemos visto en notebooks anteriores:

Vamos a revisar algunas preguntas y sus clases:

In [ ]:

import random as rn
size_training = len(dict_dataset['train'])
for i in range(10):
    index_random=rn.randint(0,size_training)
    random_instance = dict_dataset['train'][index_random]
    print(index_random, random_instance['text'], TARGET_LABELS[random_instance['label']])


2097 Where is the Henry Ford Museum ? LOC
1929 What is glass made of ? ENTY
5171 What U.S. city is known as The Rubber Capital of the World ? LOC
5210 How many times more than 3 NUM
2392 What first-aid product `` Helps the hurt stop hurting '' ? ENTY
1850 Where does Mother Angelica live ? LOC
456 What English word comes from the Old French covrefeu , meaning cover fire ? ENTY
443 What does G.M.T. stand for ? ABBR
390 What was the name of the lawyer who represented Randy Craft ? HUM
4635 Where are the busiest Amtrak rail stations in the U.S. ? LOC


### Crear un split para validación. 
Como el dataset únicamente proporciona dos splits, vamos a tomar el 10% del training como conjunto de validación. 

In [ ]:
aux = dict_dataset['train'].train_test_split(test_size=0.1)
aux

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4906
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 546
    })
})

In [ ]:
dict_dataset['train']=aux['train']
dict_dataset['val']=aux['test']
del(aux)
dict_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4906
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 546
    })
})

### Tokenización

Debemos preparar los textos para que tener el formato necesario para la entrada del transformer. Para ello deberemos utilizar el tokenizador asociado al modelo que vayamos a utilizar. 
Algunos modelos ya tienen clases predefinida para su tokenizador y para tareas específias. Por ejemplo, para cargar el tokenizador de **BERT** es posible utilizar la clase **BertTokenizer**. HuggingFace proporciona una clase que te permite cargar cualquier tokenizador, indicando simplemente el nombre del modelo. Esta clase es **AutoTokenizer**.

En este notebook, vamos a utilizar el modelo **bert-base-uncased**. 

In [ ]:
from transformers import AutoTokenizer
model_name="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


Antes de aplciar el tokenizador a todo el dataset, vamos a aplicarlo a un texto:

In [ ]:
sentence = dict_dataset['train'][0]['text']
print(sentence)
print()
encoding = tokenizer(sentence)
encoding

Where is the Bulls basketball team based ?



{'input_ids': [101, 2073, 2003, 1996, 12065, 3455, 2136, 2241, 1029, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Necesitamos conocer la longitud máximas de las oraciones en el training. En función de esta longitud máxima, podemos establecer el parámetro **MAX_LENGTH** que usaremos para representar las oraciones de nuestro dataset. Todas las oraciones tendrán la misma longitud. 
Si la longitud máxima es superior a 512 tokens, la limitaremos a 512, porque el tamaño máximo que acepta el modelo BERT es 512.  

In [ ]:
MAX_LENGTH= max([len(tokenizer(text).input_ids) for text in dict_dataset['train']['text']])
print("La longitud máxima de la secuencia es: ", MAX_LENGTH)

MAX_LENGTH = min(512, MAX_LENGTH)
print("max_length", MAX_LENGTH)


La longitud máxima de la secuencia es:  41
max_length 41


Vamos a crear una función que nos permita aplicar el tokenizador por lotes, lo que permitirá hacerlo en menos tiempo. 
El método **map** de Datasets nos permitirá aplicarlo a todo el dataset:



In [ ]:
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)


data_encodings=dict_dataset.map(tokenize, batched=True)
data_encodings

Map:   0%|          | 0/4906 [00:00<?, ? examples/s]

Map:   0%|          | 0/546 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4906
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 500
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 546
    })
})

Fijate en las longitudes de las secuencias una vez codificadas. Todas tienes la misma longitud.  

### Ejercicio: 
Prueba la funcion **tokenize** con los siguientes cambios:
- ¿Por qué si quitas truncation no hay ninguna modificación en la longitud de las oraciones?
- No uses el atributo padding, ¿cuál es la longitud de las oraciones?
- No uses el atributo max_length, ¿cuál es la longitud de las oraciones?


## Modelo 

Los textos ya están preparados!!!

Ahora debemos entrenar el modelo pre-entrenado para nuestra tarea. 

Para entrenar, podemos elegir dos framworks concretos: **tensorflow** o **pytorch**.  En este notebook, usaremos **pytorch**, que en general es más sencillo que **tensorflow**.

**Pytorch** proporciona una clase **AutoModelForSequenceClassification**, que ya carga un  transformer adaptado para dicha tarea. Esta clase permite cargar cualquier transformer, aunque muchos de ellos ya incluyen una clase específica para esta tarea. Por ejemplo, para BERT, es posible utilizar la clase **BertForSequenceClassification**. En realidad, **AutoModelForSequenceClassification** llama a la clase **BertForSequenceClassification**.

En esta clase, únicamente es necesario indicar el nombre del transformer, y el **número de clases**, en el argumento, **num_labels**. En nuestro caso, es 6.  


In [ ]:
from transformers import AutoModelForSequenceClassification

TARGET_LABELS = dict_dataset['train'].features['label'].names
NUM_LABELS = len(TARGET_LABELS)

print('TARGET_LABELS:', TARGET_LABELS, 'num_labels:', NUM_LABELS )

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS) 

TARGET_LABELS: ['ABBR', 'ENTY', 'DESC', 'HUM', 'LOC', 'NUM'] num_labels: 6


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

La ejecución de la celda anterior Produce un warning informando que algunos de los pesos pre-entrenados no están siendo utilizados y algunos han sido inicializados aleatoriamente. ¡No te preocupes, esto es completamente normal!

Los vectores finales del modelo pre-entrenado BERT es sustituido por nuevo encabezado para la clasificación que ha sido inicializado aleatoriamente. Durante el entrenamiento, este vectores se ajustan para la tarea de clasificación, transfiriéndole el conocimiento del modelo pre-entrenado. entrenado.

#### Hyperparameteres

Necesitamos definir los hiperparámetros que se emplearán durante el proceso de entrenamiento. Algunos de estos hiperpárametros son el número de epochs para el entrenamiento, el tamaño del lote (batch), learning rate, etc.  Puedes experimentar con estos hiperparámetros para encontrar la configuración óptima para tu tarea. 

En nuestro caso, trabajaremos con los hipérparametros por defecto. Necesitamos crear un objeto de la clase TrainingArguments, que incluye todos los hiperparámetros (ya inicializados con valores por defecto) que puedes ajustar. 

In [ ]:
from transformers import TrainingArguments
args = TrainingArguments(output_dir="./outputs")
# args

Vamos a modificar algunos de ellos, por ejemplo, el tamaño del batch tanto para training como para evaluación (de 8 a 32). Además, vamos a especificar la estrategia de evaluación a epoch para que nos proporcione los resultados después de cada epoch. 


In [ ]:
args.evaluation_strategy="epoch"
args.per_device_train_batch_size = 32
args.per_device_eval_batch_size = 32


#### Métricas

También tenemos que definir el conjunto de métricas que se utilizaran para evaluar el modelo sobre el conjunto de validación. Este conjunto de métricas depende de cada tarea. En el caso de la clasificación de textos, además del accuracy, es interesante conocer la precisión, recall y f1. 

Vamos a definir una función que compute estas métricas. 


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    """recibe un lote prediciones inferidas por el modelo. """
    y_true = pred.label_ids # son las labels en el gold standard
    y_pred = pred.predictions.argmax(-1) # pred.predictions devuelve una lista con las predicciones
                                        # para casda clase. Debemos quedarnos con la de mayor probabilidad.

    # como son varias clases, utilizaremos la macro
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    acc = accuracy_score(y_true, y_pred)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

#### Trainer

Ya estamos listos para entrenar el modelo. Para ello, Pytorch ya nos proporciona una clase **Trainer**,  que está optimizada para entrenar transformers, y que además nos va a ahorrar mucho trabajo (no será necesario escribir el ciclo de entrenamiento por epochs y calcular las métricas sobre el conjunto de validación). 

Para crear este objeto Trainer, deberemos pasarle el modelo, los argumentos, la función para compuar las métricas, y el conjunto de entrenamiento y validación.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,            # modelo que será ajustado
    args = args,     # hiperparámetros
    train_dataset=data_encodings['train'], # conjunto training
    eval_dataset=data_encodings['val'],   # conjunto de validación
    compute_metrics=compute_metrics,    # función para computar las métricas
)

Por fin, podemos entrenar para ajustar el modelo a la tarea de clasificación de textos: 

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4906
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 462
  Number of trainable parameters = 109486854


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.230648,0.937729,0.904296,0.924199,0.892478
2,No log,0.235771,0.945055,0.917369,0.957882,0.898193
3,No log,0.246964,0.943223,0.931148,0.957823,0.915403


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 546
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 546
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 546
  Batch size = 32


Training compl

TrainOutput(global_step=462, training_loss=0.23365841489849667, metrics={'train_runtime': 132.0423, 'train_samples_per_second': 111.464, 'train_steps_per_second': 3.499, 'total_flos': 310111154677368.0, 'train_loss': 0.23365841489849667, 'epoch': 3.0})

Una vez finalizado el entrenamiento, es posible evaluar el modelo final sobre el conjunto de datos de validación. Las métricas obtenidas son: 

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 546
  Batch size = 32


{'eval_loss': 0.24696356058120728,
 'eval_accuracy': 0.9432234432234432,
 'eval_f1': 0.9311484520950476,
 'eval_precision': 0.9578225504847824,
 'eval_recall': 0.9154029438532766,
 'eval_runtime': 1.7654,
 'eval_samples_per_second': 309.275,
 'eval_steps_per_second': 10.196,
 'epoch': 3.0}

## Evaluación

Por último, vamos a utilizar el modelo para predecir las clases para textos que no han sido utilizados durane el entrenamiento. Es decir, vamos a aplicar el modelo sobre el conjunto test.

La siguiente función recibe un texto y devuelve la clase inferida por el modelo. La función codifica el texto usando el tokenizador y el modelo es aplicado sobre esta codificación. Sobre la salida del modelo, aplicaremos una función softmax, que calcule la probabilidad de cada clase. Finalmente, devolvemos la clase con mayor probabilidad (usaremos la función **argmax** .


In [ ]:
def get_prediction(text):
    # prepara el texto, aplicamos la misma tokenización que la utilizada en el training
    inputs = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt").to("cuda")
    # aplicamos el modelo
    outputs = model(**inputs)
    # obtenemos la probabilidad para cada clase
    probs = outputs[0].softmax(1)
    # argmax nos devuelve la clase con mayor probabilidad.
    # argmax devuelve un tensor. Debemos devolver su valor asociado 
    return probs.argmax().item()

Vamos a aplicar la función **get_prediction** al conjunto de evaluación:

In [ ]:
y_pred=[get_prediction(text) for text in dict_dataset['test']['text']]


Mostramos los resultados finales. 

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=dict_dataset['test']['label'], y_pred=y_pred, target_names=TARGET_LABELS))

              precision    recall  f1-score   support

        ABBR       1.00      0.89      0.94         9
        ENTY       0.98      0.91      0.95        94
        DESC       0.97      1.00      0.98       138
         HUM       1.00      0.98      0.99        65
         LOC       0.98      0.98      0.98        81
         NUM       0.97      1.00      0.99       113

    accuracy                           0.98       500
   macro avg       0.98      0.96      0.97       500
weighted avg       0.98      0.98      0.98       500



## Práctica 1: 
Explorar los mdoelos proporcionados por HuggingFace (https://huggingface.co/models), y busca algunos modelos que puedas ajustar para la tarea de clasificción de textos en inglés. Por ejemplo, algunos poddrían ser:
- bert-base-cased
- distilbert-base-uncased
- albert-base-v2
- xlm-roberta-base

Compara los resultados de los modelos. Basta con que cambies el valor de la vairable **model_name** y vuelvas a ejecutar.

¿Qué modelo obteine mejor macro F1?, ¿cuál es más rápido?


## Práctica 2: 

Usa lo aprendido en este notebook para ajustar un modelo transformer a la tarea EXIST 2022, tanto en clasificación binaria y multiclasificación.
¿Qué resultados obtienes?, ¿cuál es el mejor modelo?

## Conclusiones

Después de este notebook, ya estarías listo para utilizar muchos de los modelos transformers (especialmente los basados en BERT) en la competición en la que vas a competir y ser evaluados en la asignatura. 

Lleva un registro de los resultados (por ejemplo, en un excel) para poder comparar los modelos, y elegir el mejor o mejores, utilizados para generar las submissions en la competición.

